In [3]:
feedback_produto = """
O XYZ Pro é um smartphone que se destaca pelo seu desempenho equilibrado,
oferecendo um processador eficiente, uma tela de alta qualidade e um bom conjunto de câmeras.
Sua bateria tem autonomia satisfatória, suportando um dia de uso moderado. 
O design é moderno, mas pode não inovar muito em relação aos concorrentes.
A interface do sistema é fluida, embora possa vir com alguns aplicativos pré-instalados desnecessários. 
No geral, é uma opção sólida para quem busca um aparelho confiável sem pagar um preço premium, 
mas pode não surpreender usuários mais exigentes.
"""

In [4]:
from langchain.prompts import ChatPromptTemplate

review_template = ChatPromptTemplate.from_template("""
para o texto a seguir,extraia as seguintes informações:
produto: nome do produto mencionado no texto.
caracteristicas_positivas: liste todas as carecteristicas positivas mencionadas sobre o produto.
caracteristicas_negativas: liste todas as caracteristicas negativas mencionadas sobre o produto.
recomendaçãp: o cliente recomenda o produto? responda True para sim ou False para não.

texto: {review}
retorne a resposta no formato JSON.                                                                                                                                                                                                                                                             
""")

In [5]:
print(review_template.format_messages(review=feedback_produto))

[HumanMessage(content='\npara o texto a seguir,extraia as seguintes informações:\nproduto: nome do produto mencionado no texto.\ncaracteristicas_positivas: liste todas as carecteristicas positivas mencionadas sobre o produto.\ncaracteristicas_negativas: liste todas as caracteristicas negativas mencionadas sobre o produto.\nrecomendaçãp: o cliente recomenda o produto? responda True para sim ou False para não.\n\ntexto: \nO XYZ Pro é um smartphone que se destaca pelo seu desempenho equilibrado,\noferecendo um processador eficiente, uma tela de alta qualidade e um bom conjunto de câmeras.\nSua bateria tem autonomia satisfatória, suportando um dia de uso moderado. \nO design é moderno, mas pode não inovar muito em relação aos concorrentes.\nA interface do sistema é fluida, embora possa vir com alguns aplicativos pré-instalados desnecessários. \nNo geral, é uma opção sólida para quem busca um aparelho confiável sem pagar um preço premium, \nmas pode não surpreender usuários mais exigentes.\

In [6]:
from langchain_openai.chat_models import ChatOpenAI
import os
import dotenv

dotenv.load_dotenv()

chat = ChatOpenAI(api_key=os.getenv("openaiKey"))
chat.invoke(review_template.format_messages(review=feedback_produto))

AIMessage(content='{\n    "produto": "XYZ Pro",\n    "caracteristicas_positivas": [\n        "Desempenho equilibrado",\n        "Processador eficiente",\n        "Tela de alta qualidade",\n        "Bom conjunto de câmeras",\n        "Bateria com autonomia satisfatória",\n        "Design moderno"\n    ],\n    "caracteristicas_negativas": [\n        "Pouca inovação em relação aos concorrentes",\n        "Alguns aplicativos pré-instalados desnecessários",\n        "Não surpreende usuários mais exigentes"\n    ],\n    "recomendacao": false\n}', response_metadata={'token_usage': {'completion_tokens': 139, 'prompt_tokens': 270, 'total_tokens': 409, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-78763480-6009-4606-9759-971af01f

In [7]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [13]:
schema_produto = ResponseSchema(
    name="produto",
    type="string",
    description="nome do produto mencionado no texto"
)

schema_positivas = ResponseSchema(
    name="caracteristicas_positivas",
    type="list",
    description="liste todas as carecteristicas positivas mencionadas sobre o produto"
)


schema_negativas = ResponseSchema(
    name="caracteristicas_negativas",
    type="list",
    description="liste todas as carecteristicas negativas mencionadas sobre o produto"
)


schema_recomendacao = ResponseSchema(
    name="recomendacao",
    type="bool",
    description="o cliente recomenda o produto?True para sim ou False para não"
)

In [14]:
response_schema = [schema_produto,schema_positivas,schema_negativas,schema_recomendacao]


output_parser = StructuredOutputParser.from_response_schemas(response_schema)
schema_formatado = output_parser.get_format_instructions()

In [15]:
print(schema_formatado)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"produto": string  // nome do produto mencionado no texto
	"caracteristicas_positivas": list  // liste todas as carecteristicas positivas mencionadas sobre o produto
	"caracteristicas_negativas": list  // liste todas as carecteristicas negativas mencionadas sobre o produto
	"recomendacao": bool  // o cliente recomenda o produto?True para sim ou False para não
}
```


In [16]:
review_template2 = ChatPromptTemplate.from_template("""
para o texto a seguir,extraia as seguintes informações:
produto,caracteristicas_positivas,caracteristicas_negativas,recomendacao

texto:{review}
{schema}                                          
""",
partial_variables={"schema":schema_formatado})

In [17]:
print(review_template2.format_messages(review=feedback_produto))

[HumanMessage(content='\npara o texto a seguir,extraia as seguintes informações:\nproduto,caracteristicas_positivas,caracteristicas_negativas,recomendacao\n\ntexto:\nO XYZ Pro é um smartphone que se destaca pelo seu desempenho equilibrado,\noferecendo um processador eficiente, uma tela de alta qualidade e um bom conjunto de câmeras.\nSua bateria tem autonomia satisfatória, suportando um dia de uso moderado. \nO design é moderno, mas pode não inovar muito em relação aos concorrentes.\nA interface do sistema é fluida, embora possa vir com alguns aplicativos pré-instalados desnecessários. \nNo geral, é uma opção sólida para quem busca um aparelho confiável sem pagar um preço premium, \nmas pode não surpreender usuários mais exigentes.\n\nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"produto": string  // nome do produto mencionado no texto\n\t"caracteristicas_positivas": list  // lis

In [18]:
resposta = chat.invoke(review_template2.format_messages(review=feedback_produto))

In [19]:
resposta.content

'```json\n{\n\t"produto": "XYZ Pro",\n\t"caracteristicas_positivas": [\n\t\t"desempenho equilibrado",\n\t\t"processador eficiente",\n\t"tela de alta qualidade",\n\t"bom conjunto de câmeras",\n\t"bateria com autonomia satisfatória",\n\t"design moderno"\n\t],\n\t"caracteristicas_negativas": [\n\t\t"design pode não inovar muito em relação aos concorrentes",\n\t\t"alguns aplicativos pré-instalados desnecessários"\n\t],\n\t"recomendacao": true\n}\n```'

In [21]:
resposta_json = output_parser.parse(resposta.content)

In [22]:
resposta_json["produto"]

'XYZ Pro'